# Lab 3: Gesture Recognition using Convolutional Neural Networks

In this lab you will train a convolutional neural network to make classifications on different hand gestures. By the end of the lab, you should be able to:

1. Load and split data for training, validation and testing
2. Train a Convolutional Neural Network
3. Apply transfer learning to improve your model

Note that for this lab we will not be providing you with any starter code. You should be able to take the code used in previous labs, tutorials and lectures and modify it accordingly to complete the tasks outlined below.

### What to submit

Submit a PDF file containing all your code, outputs, and write-up
from parts 1-5. You can produce a PDF of your Google Colab file by
going to **File > Print** and then save as PDF. The Colab instructions
has more information. Make sure to review the PDF submission to ensure that your answers are easy to read. Make sure that your text is not cut off at the margins.

**Do not submit any other files produced by your code.**

Include a link to your colab file in your submission.

Please use Google Colab to complete this assignment. If you want to use Jupyter Notebook, please complete the assignment and upload your Jupyter Notebook file to Google Colab for submission.

## Colab Link

Include a link to your colab file here

Colab Link: https://colab.research.google.com/github/WilliamJWen/APS360/blob/main/lab/Lab3_Gesture_Recognition.ipynb

## Dataset

American Sign Language (ASL) is a complete, complex language that employs signs made by moving the
hands combined with facial expressions and postures of the body. It is the primary language of many
North Americans who are deaf and is one of several communication options used by people who are deaf or
hard-of-hearing. The hand gestures representing English alphabet are shown below. This lab focuses on classifying a subset
of these hand gesture images using convolutional neural networks. Specifically, given an image of a hand
showing one of the letters A-I, we want to detect which letter is being represented.

![alt text](https://www.disabled-world.com/pics/1/asl-alphabet.jpg)

## Part B. Building a CNN [50 pt]

For this lab, we are not going to give you any starter code. You will be writing a convolutional neural network
from scratch. You are welcome to use any code from previous labs, lectures and tutorials. You should also
write your own code.

You may use the PyTorch documentation freely. You might also find online tutorials helpful. However, all
code that you submit must be your own.

Make sure that your code is vectorized, and does not contain obvious inefficiencies (for example, unecessary
for loops, or unnecessary calls to unsqueeze()). Ensure enough comments are included in the code so that
your TA can understand what you are doing. It is your responsibility to show that you understand what you
write.

**This is much more challenging and time-consuming than the previous labs.** Make sure that you
give yourself plenty of time by starting early.

### 1. Data Loading and Splitting [5 pt]

Download the anonymized data provided on Quercus. To allow you to get a heads start on this project we will provide you with sample data from previous years. Split the data into training, validation, and test sets.

Note: Data splitting is not as trivial in this lab. We want our test set to closely resemble the setting in which
our model will be used. In particular, our test set should contain hands that are never seen in training!

Explain how you split the data, either by describing what you did, or by showing the code that you used.
Justify your choice of splitting strategy. How many training, validation, and test images do you have?

For loading the data, you can use plt.imread as in Lab 1, or any other method that you choose. You may find
torchvision.datasets.ImageFolder helpful. (see https://pytorch.org/docs/stable/torchvision/datasets.html?highlight=image%20folder#torchvision.datasets.ImageFolder
)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os
import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import DataLoader, Subset
from torch.utils.data.sampler import SubsetRandomSampler
import random

import torch.nn as nn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/APS360/lab3

/content/drive/MyDrive/APS360/lab3


In [ ]:
# globals
seed = 117
random.seed(seed)

torch.manual_seed(seed)

In [ ]:
dataset_path = 'Lab3_Gestures_Summer'

# transformation process
transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load dataset
dataset = datasets.ImageFolder(dataset_path, transform=transform)

def get_prefix_num(filename):
  return int(filename.split("_")[0])
def get_suffix_num(filename):
  return int(filename.split("_")[-1].split(".")[0])

In [ ]:
prefix_seen = set()
duplicate = False

for img_path, label in dataset.imgs:
  filename = os.path.basename(img_path)
  prefix_id = get_prefix_num(filename)
  if prefix_id in prefix_seen:
    duplicate = True
    break

if (duplicate):
  print("There is duplicate prefix id.")
else:
  print("All prefix id is unique confirmed")

All prefix id is unique confirmed


In [ ]:
### put images into groups of same person and light

# store original indices
indexed_images = [(i, img_path, label) for i, (img_path, label) in enumerate(dataset.imgs)]
sorted_images = sorted(indexed_images, key=lambda x: get_prefix_num(os.path.basename(x[1])))

groups = []
current_group = []

for og_idx, img_path, label in sorted_images:
  filename = os.path.basename(img_path)
  prefix_id = get_prefix_num(filename)

  if not current_group:
    current_group.append(og_idx)
  else:
    prev_filename = os.path.basename(indexed_images[current_group[-1]][1])
    prev_prefix = get_prefix_num(prev_filename)

    # group sequential images into a group
    if prefix_id == prev_prefix + 1 and get_suffix_num(filename) == get_suffix_num(prev_filename) + 1:
      current_group.append(og_idx)
    else:
      # start a new group
      groups.append(current_group)
      current_group = [og_idx]

# append last group
if current_group:
  groups.append(current_group)

In [ ]:
print(groups)

[[118, 172], [429, 442, 455], [707, 720, 736], [992, 758, 774], [1026, 1036, 1049], [1302, 1314, 1329], [1583, 1593, 1609], [1858, 1871, 1884], [2131, 2132, 2136], [166, 167, 171], [418, 419, 423], [666, 667, 670], [917, 918, 922], [1168, 1169, 1173], [1407, 1408, 1412], [1662, 1663, 1667], [1910, 1911, 1915], [2162, 2163, 2167], [200], [204], [232, 233, 237], [483, 484, 488], [730, 731, 735], [980, 981, 985], [1231, 1232, 1236], [1470, 1471, 1475], [1725, 1476, 1480], [1729, 1730, 1734], [1976, 1977, 1981], [11, 12, 16], [269, 270, 274], [517, 518, 522], [768, 769, 773], [1017, 1018, 1022], [1266, 1267, 1268], [1508, 1509, 1513], [1759, 1760, 1764], [2007, 2008, 2012], [44, 45, 49], [302], [547, 548, 552], [797, 798, 799], [1043, 1044, 1048], [1290, 1291, 1295], [1538, 1539, 1543], [1789, 1790, 1794], [2037, 2038, 2042], [74, 75, 79], [328, 329, 333], [577, 578, 582], [827, 828, 832], [1077, 1078, 1082], [1323, 1324, 1328], [1571, 1572, 1576], [1822, 1823, 1824], [2070, 2071, 2075], [

In [ ]:
# randomly split into train, val and test
random.shuffle(groups)

train_ratio, val_ratio = 0.7, 0.15
num_groups = len(groups)
train_end = int(train_ratio * num_groups)
val_end = train_end + int(val_ratio * num_groups)

# split the groups
train_groups = groups[:train_end]
val_groups = groups[train_end:val_end]
test_groups = groups[val_end:]

In [ ]:
# collect indices from groups
train_indices = [idx for group in train_groups for idx in group]
val_indices = [idx for group in val_groups for idx in group]
test_indices = [idx for group in test_groups for idx in group]


# Subset Datasets
train_set = Subset(dataset, train_indices)
val_set = Subset(dataset, val_indices)
test_set = Subset(dataset, test_indices)

# DataLoaders
batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

print(f"Training set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Testset size: {len(test_set)}")


Training set size: 1560
Validation set size: 330
Testset size: 329


Answer:
- I first put all images into groups of images that have the same hand and lightning condition. And shuffled and splited the groups into 70-15-15 ratio for test-validation and test respectively. And then unpack the groups to images to load.

- In this way, all images in validation and test would be hands that are not in test accuracy, so that they can accurately simulate model's performance when working with new data in real world.

- Also, this global random shuffle on groups would work, because we have a balanced dataset where each class has around 266 images and each group has almost exact three images.



### 2. Model Building and Sanity Checking [15 pt]

### Part (a) Convolutional Network - 5 pt

Build a convolutional neural network model that takes the (224x224 RGB) image as input, and predicts the gesture
letter. Your model should be a subclass of nn.Module. Explain your choice of neural network architecture: how
many layers did you choose? What types of layers did you use? Were they fully-connected or convolutional?
What about other decisions like pooling layers, activation functions, number of channels / hidden units?

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.name = "cnn"
    self.conv1 == nn.Conv2d(3, 5, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(5, 10, 5)



### Part (b) Training Code - 5 pt

Write code that trains your neural network given some training data. Your training code should make it easy
to tweak the usual hyperparameters, like batch size, learning rate, and the model object itself. Make sure
that you are checkpointing your models from time to time (the frequency is up to you). Explain your choice
of loss function and optimizer.

### Part (c) “Overfit” to a Small Dataset - 5 pt

One way to sanity check our neural network model and training code is to check whether the model is capable
of “overfitting” or “memorizing” a small dataset. A properly constructed CNN with correct training code
should be able to memorize the answers to a small number of images quickly.

Construct a small dataset (e.g. just the images that you have collected). Then show that your model and
training code is capable of memorizing the labels of this small data set.

With a large batch size (e.g. the entire small dataset) and learning rate that is not too high, You should be
able to obtain a 100% training accuracy on that small dataset relatively quickly (within 200 iterations).

### 3. Hyperparameter Search [15 pt]

### Part (a) - 3 pt

List 3 hyperparameters that you think are most worth tuning. Choose at least one hyperparameter related to
the model architecture.

### Part (b) - 5 pt

Tune the hyperparameters you listed in Part (a), trying as many values as you need to until you feel satisfied
that you are getting a good model. Plot the training curve of at least 4 different hyperparameter settings.

### Part (c) - 3 pt
Choose the best model out of all the ones that you have trained. Justify your choice.

### Part (d) - 4 pt
Report the test accuracy of your best model. You should only do this step once and prior to this step you should have only used the training and validation data.

### 4. Transfer Learning [15 pt]
For many image classification tasks, it is generally not a good idea to train a very large deep neural network
model from scratch due to the enormous compute requirements and lack of sufficient amounts of training
data.

One of the better options is to try using an existing model that performs a similar task to the one you need
to solve. This method of utilizing a pre-trained network for other similar tasks is broadly termed **Transfer
Learning**. In this assignment, we will use Transfer Learning to extract features from the hand gesture
images. Then, train a smaller network to use these features as input and classify the hand gestures.

As you have learned from the CNN lecture, convolution layers extract various features from the images which
get utilized by the fully connected layers for correct classification. AlexNet architecture played a pivotal
role in establishing Deep Neural Nets as a go-to tool for image classification problems and we will use an
ImageNet pre-trained AlexNet model to extract features in this assignment.

### Part (a) - 5 pt
Here is the code to load the AlexNet network, with pretrained weights. When you first run the code, PyTorch
will download the pretrained weights from the internet.

In [ ]:
import torchvision.models
alexnet = torchvision.models.alexnet(pretrained=True)

The alexnet model is split up into two components: *alexnet.features* and *alexnet.classifier*. The
first neural network component, *alexnet.features*, is used to compute convolutional features, which are
taken as input in *alexnet.classifier*.

The neural network alexnet.features expects an image tensor of shape Nx3x224x224 as input and it will
output a tensor of shape Nx256x6x6 . (N = batch size).

Compute the AlexNet features for each of your training, validation, and test data. Here is an example code
snippet showing how you can compute the AlexNet features for some images (your actual code might be
different):

In [ ]:
# img = ... a PyTorch tensor with shape [N,3,224,224] containing hand images ...
features = alexnet.features(img)

**Save the computed features**. You will be using these features as input to your neural network in Part
(b), and you do not want to re-compute the features every time. Instead, run *alexnet.features* once for
each image, and save the result.

### Part (b) - 3 pt
Build a convolutional neural network model that takes as input these AlexNet features, and makes a
prediction. Your model should be a subclass of nn.Module.

Explain your choice of neural network architecture: how many layers did you choose? What types of layers
did you use: fully-connected or convolutional? What about other decisions like pooling layers, activation
functions, number of channels / hidden units in each layer?

Here is an example of how your model may be called:

In [ ]:
# features = ... load precomputed alexnet.features(img) ...
output = model(features)
prob = F.softmax(output)

### Part (c) - 5 pt
Train your new network, including any hyperparameter tuning. Plot and submit the training curve of your
best model only.

Note: Depending on how you are caching (saving) your AlexNet features, PyTorch might still be tracking
updates to the **AlexNet weights**, which we are not tuning. One workaround is to convert your AlexNet
feature tensor into a numpy array, and then back into a PyTorch tensor.

In [ ]:
tensor = torch.from_numpy(tensor.detach().numpy())

### Part (d) - 2 pt
Report the test accuracy of your best model. How does the test accuracy compare to Part 3(d) without transfer learning?